In [1]:
import pandas as pd
import math
import numpy as np

import heapq
import numba as nb
from scipy.stats.mstats import gmean, hmean
from state import*

data_full = pd.read_csv('data_test.csv')
data_full = data_full.sort_values(by=['TIME', 'PROFIT'], ascending=[False, False], ignore_index=True)
all_fomula = np.array(pd.read_csv('congthuc.csv')['fomula'])

data_full

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Cash and cash equivalents,Short-term trade payables,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation
0,90,0.000000,AAA,HOSE,4.893114e+12,0.0,3.926632e+09,5.918750e+10,5156710.0,6.215189e+10,...,1.018940e+12,9.794080e+11,2.943622e+10,3.430300e+11,2.816613e+11,3.589115e+10,3.873387e+09,1.870343e+10,5.338777e+12,-1.579764e+12
1,90,0.000000,AAT,HOSE,8.485598e+11,1384.0,1.576288e+08,5.314787e+09,1027330.0,9.241194e+07,...,3.328005e+10,1.410469e+10,2.253305e+10,1.191474e+11,2.649353e+06,5.314787e+09,0.000000e+00,0.000000e+00,3.428906e+11,-1.813202e+11
2,90,0.000000,ABS,HOSE,9.451310e+11,0.0,6.931579e+08,1.141712e+10,737770.0,8.641370e+07,...,2.160073e+10,1.738495e+11,2.044294e+10,2.653042e+10,1.491120e+09,0.000000e+00,0.000000e+00,2.410694e+09,7.281878e+11,-7.332759e+09
3,90,0.000000,ACC,HOSE,1.911000e+12,258.0,9.234700e+04,5.818817e+09,324230.0,2.771447e+10,...,7.737720e+11,8.828534e+10,8.608350e+09,1.338862e+10,8.726839e+10,5.818817e+09,0.000000e+00,1.173808e+10,8.545696e+11,-1.158529e+11
4,90,0.000000,ACL,HOSE,1.040800e+12,673.0,1.132181e+09,1.032095e+10,309440.0,3.054674e+09,...,8.086112e+10,3.258525e+10,1.426303e+10,7.655423e+10,4.100863e+09,1.022072e+10,3.327546e+09,9.132208e+09,7.208971e+11,-4.303955e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7057,29,0.829231,GIL,HOSE,2.957500e+11,0.0,1.178573e+07,4.434805e+08,24353.0,1.918037e+08,...,5.592835e+09,2.042547e+10,1.471138e+08,1.327977e+10,9.091083e+08,2.843084e+08,0.000000e+00,2.477158e+08,4.939735e+10,-1.132837e+10
7058,29,0.792308,VSH,HOSE,8.125000e+12,0.0,0.000000e+00,4.036377e+09,80405.0,1.217429e+10,...,1.117020e+11,7.003186e+09,4.384513e+09,5.236953e+10,3.844896e+08,3.997361e+09,0.000000e+00,3.763020e+09,5.471286e+11,-1.213501e+12
7059,29,0.741068,FPT,HOSE,2.961458e+13,2550.0,6.714888e+09,1.364417e+10,101680.0,1.010007e+10,...,5.111802e+11,4.444050e+11,1.211761e+11,4.029315e+11,7.813338e+10,9.609989e+09,0.000000e+00,2.484622e+10,1.477624e+12,-2.382139e+11
7060,29,0.724551,GMD,HOSE,7.643971e+12,0.0,1.093278e+07,9.889580e+09,132220.0,2.829137e+10,...,1.335678e+12,0.000000e+00,0.000000e+00,5.367310e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.744527e+11,-2.710334e+11


In [2]:
result, per_file = normal_main(player_random, all_fomula,  1,[0])
result

['(-A+T*Y/E)/A' '(+N*O*X*D)/A/A/A']
check 2 28.0 quarter 0
quarter 0 check 0.5 action 2 topaction [18. 28. 28.]
check 1 14.0 quarter 1
quarter 1 check 1.0 action 1 topaction [14. 36. 44.]
check 1 54.0 quarter 2
quarter 2 check 0.3333333333333333 action 1 topaction [ 8. 34. 54.]
check 0 2 quarter 3
quarter 3 check 1.0 action 0 topaction [ 2. 21. 26.]
check 1 35.0 quarter 4
quarter 4 check 0.3333333333333333 action 1 topaction [ 6. 10. 35.]
check 1 40.0 quarter 5
quarter 5 check 0.3333333333333333 action 1 topaction [ 3.  6. 40.]
check 2 15.0 quarter 6
quarter 6 check 1.0 action 2 topaction [ 1.  4. 15.]
check 1 18.0 quarter 7
quarter 7 check 0.5 action 1 topaction [ 3. 18. 19.]
check 2 68.0 quarter 8
quarter 8 check 1.0 action 2 topaction [26. 67. 68.]
check 2 79.0 quarter 9
quarter 9 check 1.0 action 2 topaction [73. 79. 86.]
check 0 14 quarter 10
quarter 10 check 1.0 action 0 topaction [ 14.  99. 135.]
check 2 35.0 quarter 11
quarter 11 check 1.0 action 2 topaction [33. 35. 86.]
check

array([0.6010913])

In [6]:
LIST_RANK_CT1, LIST_COMP1 = get_in4_rank_fomula('(-A+T*Y/E)/A')
LIST_RANK_CT2, LIST_COMP2 = get_in4_rank_fomula('(+N*O*X*D)/A/A/A')
data_full['(-A+T*Y/E)/A'] = np.nan_to_num(eval('(-A+T*Y/E)/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)
data_full['(+N*O*X*D)/A/A/A'] = np.nan_to_num(eval('(+N*O*X*D)/A/A/A'), nan=-math.inf, posinf=-math.inf, neginf=-math.inf)


In [9]:
quarter = 7
LIST_RANK_NOT_INVEST[quarter], LIST_RANK_CT1[quarter], LIST_COMP1[quarter], LIST_RANK_CT2[quarter], LIST_COMP2[quarter]

(19, 18, 'PVD', 3, 'ITA')

In [10]:
test = data_full[data_full['TIME'] == 29+quarter].reset_index(drop=True)
test.head(30)

,TIME,PROFIT,SYMBOL,EXCHANGE,MARKET_CAP,Basic earnings per share,Other expenses,Financial expenses,VolumeARG,Financial income,...,Taxes and other obligations to the State Budget,Gross profit,Other short-term payables,In which: Loan interest expenses,Sales deductions,Short-term accrued expenses,LIABILITIES,Tangible fixed assets Accumulated depreciation,(-A+T*Y/E)/A,(+N*O*X*D)/A/A/A
0,36,1.598901,PPC,HOSE,5.917814e+12,0.0,-9.815689e+09,1.582738e+12,167544.0,1.137689e+11,...,2.400504e+10,2.079311e+11,1.002437e+11,4.703013e+10,0.000000e+00,3.903138e+11,7.361025e+12,-8.141237e+12,1.543715e+06,0.000000e+00
1,36,1.557554,HPG,HOSE,5.458919e+12,0.0,1.375398e+10,4.066298e+10,162626.0,6.279491e+10,...,0.000000e+00,-1.922543e+11,0.000000e+00,1.648304e+10,4.314177e+10,0.000000e+00,1.272283e+12,0.000000e+00,-2.755274e+05,-0.000000e+00
2,36,1.537445,ITA,HOSE,3.001956e+12,521.0,3.664200e+07,1.042596e+10,69805.0,4.357307e+10,...,2.154982e+10,8.772103e+10,7.783760e+10,8.137589e+09,0.000000e+00,6.915564e+11,1.690892e+12,-1.619542e+10,7.078285e+05,1.955309e+06
3,36,1.528481,TDH,HOSE,7.584000e+11,4325.0,1.555869e+08,9.349408e+09,92334.0,1.841310e+10,...,5.066299e+10,1.528927e+11,3.509776e+09,0.000000e+00,1.051330e+09,-1.676630e+06,4.245713e+11,-2.084452e+10,9.269940e+05,-1.514918e+02
4,36,1.482759,GMD,HOSE,1.246857e+12,0.0,3.132495e+08,1.430522e+11,53506.0,5.704140e+10,...,1.040109e+10,7.174335e+10,1.315776e+11,2.844896e+10,0.000000e+00,1.994442e+10,1.468586e+12,-4.458508e+11,1.579290e+06,1.430019e+05
5,36,1.480769,MPC,HOSE,7.278091e+11,0.0,5.145630e+08,3.817001e+11,98864.0,3.456532e+11,...,1.812600e+10,1.510432e+11,7.455699e+10,4.888738e+10,5.823198e+09,0.000000e+00,1.287970e+12,-1.009024e+11,2.703653e+06,0.000000e+00
6,36,1.420000,LCG,HOSE,4.760000e+11,0.0,3.213825e+10,3.356027e+09,54087.0,3.180085e+10,...,6.912259e+10,5.666703e+10,1.240629e+11,3.301327e+09,0.000000e+00,5.134691e+09,3.658858e+11,-3.613637e+09,8.053331e+05,5.463955e+04
7,36,1.410714,KDC,HOSE,1.256230e+12,0.0,3.720568e+10,4.920723e+10,40487.0,3.467020e+10,...,4.237072e+09,8.412795e+10,1.943777e+11,1.452302e+10,1.238421e+09,3.648255e+09,8.182555e+11,-9.051696e+10,1.353456e+06,-1.056659e+03
8,36,1.366337,VIP,HOSE,6.040586e+11,-35.0,5.463808e+08,5.850284e+10,137793.0,7.135901e+09,...,0.000000e+00,5.273686e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.583569e+11,0.000000e+00,4.171270e+05,-0.000000e+00
9,36,1.251613,DPM,HOSE,1.175892e+13,0.0,4.270041e+08,1.699388e+10,186131.0,3.090193e+10,...,1.389419e+10,8.251229e+10,2.946903e+10,1.064748e+10,2.691938e+09,8.605137e+10,4.825484e+11,-3.731923e+12,1.819070e+04,-1.371551e+01


In [12]:
np.argmax(test['(-A+T*Y/E)/A'])

17

In [13]:
np.argmax(test['(+N*O*X*D)/A/A/A'])


2